### How to use this notebook (for anyone not familiar with jupyter)

Before we get started, I'd like to note that while reading this notebook on github is fine, there are several interactive elements used to demonstrate several ideas that only render when the notebook is ran in a live kernel. This also allows you to play with the code samples inline with the rest of the notebook.

For the best experience I'd recommend installing [jupyter](https://jupyter.org/install) on your machine and cloning this repository so that you can run the notebooks locally. 

If however you'd prefer not to install anything, you can also run this notebook on Google's Colab [here](https://colab.research.google.com/github/jd-13/dsp-math-for-audio/blob/master/04_Compressors.ipynb).

Once running click `Kernel` -> `Restart & Run All` (or `Runtime` -> `Run all` if you're using Colab) to render all of the interactive examples, and you are then free to edit the code and run individual cells as you wish.

In [1]:
"""
This code cell contains a few imports and functions that we'll need later but aren't of much interest.
The main content starts immediately after.
"""

from ipywidgets import interact
import matplotlib.pylab as plt
import math
import numpy as np
import scipy.signal
%matplotlib inline

# We'll use this to define the x-axis of any graphs we plot
NUM_SAMPLES = 200
SAMPLE_RANGE = np.linspace(0, NUM_SAMPLES, NUM_SAMPLES)

# Run at a lower than usual sample rate to keep the interactive examples from
# needing too much compute
SAMPLE_RATE = 10000

Y_LIMIT = 0.8

def sineWaveDecay(frequencyHz):
    """
    Returns a np array of length numSamples containing a sine wave impulse
    which decays linearly.
    """    
    # Create a square impulse
    amplitudeMap = np.zeros((NUM_SAMPLES))
    amplitudeMap[70:75] = 1
    
    # Smooth the amplitude map
    b, a = scipy.signal.butter(4, 0.1, btype="lowpass", analog=False)
    amplitudeMap = scipy.signal.lfilter(b, a, amplitudeMap)
    
    return amplitudeMap

MINUS_INF_DB = -100

def convertTodB(inputBuffer):
    outputBuffer = np.zeros(inputBuffer.shape)
    
    for index in range(len(inputBuffer)):
        if inputBuffer[index] > 0:
            outputBuffer[index] = max(20 * math.log(inputBuffer[index], 10), MINUS_INF_DB)
        else:
            outputBuffer[index] = MINUS_INF_DB
    
    return outputBuffer

def convertToLinear(inputBuffer):
    outputBuffer = np.zeros(inputBuffer.shape)

    for index in range(len(inputBuffer)):
        if inputBuffer[index] > MINUS_INF_DB:
            outputBuffer[index] = pow(10, inputBuffer[index] / 20.0)
        else:
            outputBuffer[index] = 0

    return outputBuffer

## Intro

Dynamic range compression is an important concept within music production, and is used extensively throughout the production and mixing process. Compression can be used to subtlely maintain a consistent level on a recording, or at its extremes squash a signal until it produces noticeable distortion and artefacts, among many other more creative uses.

In this notebook we'll dicuss:
- how and why a compressor might used
- the characteristics that are important to the design of a compressor and that differentiate one compressor from another
- how a compressor works (with some interactive graphics!)
- how to implement a basic compressor (with working and complete code examples)

Note that many of the code examples in this notebook are intended to be readable rather than performant, so you may wish to make changes before attempting to use these examples in your own projects.

## Overview of compressors

In the simplest case, the main goal of a compressor is to reduce the dynamic range of the audio it is processing. In other words, a compressor reduces the volume difference between the loudest parts of an audio signal and the quietest.

This is an important technique particularly for recordings such as vocals and bass guitar, which typically have a large variance in volume (the dynamic range) which from one moment to the next may be a little too loud for the mix or a little too quiet. This is where compression can be used to level out the peaks and troughs in the volume of the signal, so that it sits nicely and consistently with other elements of a mix.

There are two main ways to approach the problem of dynamic range compression:
- **Downward compression**: Reduce the volume of everything that *exceeds* a threshold
- **Upward compression**: Increase the volume of everything that falls *below* a threshold

Typically both types of compressor will then apply a "makeup" gain to the entire signal which controls the output level of the compressor.

The amount of compression that is applied to a signal above (in the case of a downward compressor) or below (in the case of an upward compressor) the threshold is determined by a *ratio* parameter.

That said, most popular compressors you are likely to see in use fall into the downward compression category, with upward compression being used very rarely.

The way a compressor applies its volume reduction can be visualised by plotting a graph of input amplitude on the x-axis and output amplitude on the y-axis as below:

In [2]:
def compressDown(threshold, ratio, inputAmp):
    if inputAmp > threshold:
        return threshold + (inputAmp - threshold) / ratio
    else:
        return inputAmp

def compressUp(threshold, ratio, inputAmp):
    if inputAmp > threshold:
        return inputAmp
    else:
        return threshold - (threshold - inputAmp) / ratio

@interact(threshold=(0.2, 0.9, 0.05), ratio=(1, 10, 1), inputAmp=(0, 1, 0.05))
def plotCompression(threshold, ratio, inputAmp):
    plt.figure(figsize=(8, 4))
    xcoords = [0, threshold, 1]
        
    # Plot downward
    ycoords = [0, threshold, compressDown(threshold, ratio, 1)]
    outputAmp = compressDown(threshold, ratio, inputAmp)

    plt.subplot(1, 2, 1)
    plt.plot(xcoords, ycoords)
    plt.plot([inputAmp, inputAmp], [0, outputAmp])
    plt.plot([0, inputAmp], [outputAmp, outputAmp])

    plt.xlim(left=0, right=1)
    plt.ylim(bottom=0, top=1)
    plt.xlabel("Input amplitude")
    plt.ylabel("Output amplitude")
    plt.title("Downward compression")
    
    # Plot upward
    ycoords = [compressUp(threshold, ratio, 0), threshold, 1]
    outputAmp = compressUp(threshold, ratio, inputAmp)

    plt.subplot(1, 2, 2)
    plt.plot(xcoords, ycoords)
    plt.plot([inputAmp, inputAmp], [0, outputAmp])
    plt.plot([0, inputAmp], [outputAmp, outputAmp])

    plt.xlim(left=0, right=1)
    plt.ylim(bottom=0, top=1)
    plt.xlabel("Input amplitude")
    plt.ylabel("Output amplitude")
    plt.title("Upward compression")

    
    plt.show()

interactive(children=(FloatSlider(value=0.55, description='threshold', max=0.9, min=0.2, step=0.05), IntSlider(value=5, description='ratio', max=10, min=1), FloatSlider(value=0.0, description='inputAmp', max=1.0, step=0.05), Output()), _dom_classes=('widget-interact',))

In the case of the downward compressor, at input volumes below a **threshold** no compression is applied, therefore the output amplitude equals the input amplitude. But at input amplitudes above the **threshold** the gain reduction is applied. The amount of gain reduction is described as a **ratio**, such as 2:1, 4:1, etc. This means that when the input amplitude exceeds the threshold, the output amplitude will be equal to everything below the threshold, *plus* everything above the threshold divided by the ratio.

For the upward compressor you can see the opposite is true, with the signal being *multiplied* by the ratio when below the threshold, but passed through unchanged when above the threshold.

But this simple dynamic range adjustment is not all that compressors are used for, with many models of analogue compressor becaming famous because of their imperfections which lead them to do a little more to a signal than just reduce its dynamic range. These imperfections may cause the compressor to affect different frequency ranges differently or introduce a small level or distortion, in turn changing the frequency content of a signal by a small amount.

Another important feature of a compressor which we've not yet mentioned is the **attack** and **release** time. These parameters control how quickly the compressor applies the volume reduction after the input amplitude has exceeded the threshold, and for how long it continues to apply the volume reduction after the input amplitude has dropped back below the threshold (or vice versa for an upwards compressor).

These controls are very important to controlling the behaviour of the compressor, for example a slower attack allows more of the initial transient of a sound to pass through before triggering the compressor, and slower release will cause the compressor to hang on to a sound for longer.

## Types of compressor


### Analogue

Until recent years, all compressors were what we would consider to be analogue compressors. This means the compressor would be a physical piece of hardware (often rackmounted), and would use an electrical circuit to apply the compression. This type of compressor is still in widespread use today, and several models of analogue compressor have become sought-after enough to warrant software emulations of them.

One of the main areas that these compressors differentiated themselves from the competition was in the implementation of their compression circuits, the most popular of which can roughly be grouped as follows:

|                | Optical                                                        | FET                                  | VCA                                                 |
| :------------: | :------------------------------------------------------------: | :----------------------------------: | :-------------------------------------------------: |
| Character      | Often described as "warm" or "colourful", noticable distortion | Less colourful, less distortion      | Usually very clean, minimal colouration             |
| Attack/Release | Typically slowest                                              | Faster                               | Often equally capable at slow and fast times        |
| Application    | Popular for vocals, melodic instruments                        | Often used on vocals, guitars, drums | Bus compressors                                     |
| As seen in     | Teletronicx LA-2A and LA-3A                                    | UREI 1176                            | Focusrite Red, dbx 160, SSL G master bus compressor |

*Some of the statements above are broad generalisations of each design, and you are likely to find examples of each which contradict these*

### Digital

In recent years digital compressors have become much more popular, due to lower cost, new design possibilties, and multiple possible form factors. One of the key advantages of a digital compressor is the it can be built not only as a rack unit, but also a VST plugin, standalone software, or integrated into a digital effects unit or pedalboard alongside dozens of other effects.

Digital compression also opens up many possibilities in terms of how a compressor can sound, since the design is no longer limited by the components which makeup the circuit but rather how much can be computed while keeping the latency low.

Because of this, in recent years all kinds of software based compressors have been created, with some aiming to be super accurate models of the previously mentioned analogue compressors (often with a few added improvements), some aiming to be the cleanest, fastest, and most transparent compressor, and others going in their own direction to reimagine what a compressor can sound like.

## Other compressor based devices

Before we go any further and start looking into the implementation of a basic compression algorithm, you may come across a few other devices that perform a similar function to a compressor which are worth briefly mentioning. This isn't an exhaustive list, but contains the most common of these devices.

### Limiters

A limiter is typically a specialised version of a downward compressor, usually with a very high ratio. These devices are often used where an audio signal needs to be prevented from significantly exceeding the threshold, and are a staple of most mastering processes.

### Multiband compressors

A multiband compressor works by first using several crossover filters to split a signal into two or more frequency bands and then passing the resulting signals to separate compressors. For example in a 3 band multiband compressor, you would have one compressor for the lowest frequencies, one compressor for those in the middle, and another for the highest.

In most implementations the compressors of each band can function independently, with their own controls for threshold, ratio, etc. The frequency range which each compressor is responsible for is often also controllable by the user.

### De-essers

Usually used to reduce sibilance in vocal recordings, a de-esser is often implemented as a compressor which only operates on a particular frequency range. In this way it can limit the amplitude of the frequencies in the range within which the sibilence is occurring by applying compression to only those frequncies.

### Expanders

An expander is effectively a device which does the opposite of a compressor, and increases the dynamic range of an audio signal rather than trying to reduce it.

## Implementation

*(Much of this section is based on the paper [Digital Dynamic Range Compressor Design - A Tutorial and Analysis](https://www.researchgate.net/publication/277772168_Digital_Dynamic_Range_Compressor_Design-A_Tutorial_and_Analysis), which I recommend reading for additional details)*

Now that we know a little about what compressors do and how they're used, we can start looking at how to actually build one. In most compressors there are two key components: the level detector, and the gain computer.

Between these two the gain reduction applied to the signal is calculated, and we'll look at how these work exactly very soon. First though we need a higher level view of how these components can be arranged, in either feedforward or feedback designs.

### Feedforward vs feedback

Both analogue and digital compressors can be implemented using feedforward or feedback designs. Below you can see a simplified diagram of the signal chain of a feedforward design:

![](https://raw.githubusercontent.com/jd-13/dsp-math-for-audio/master/assets/compressors/simpleFeedforward.png)

The process can be summarised as the following steps:
1. Input is split between being routed to the output and the level detector
2. Level detector generates a signal representing the amplitude of the input
3. Gain computer takes the output of the level detector, and checks it against the parameters (threshold, ratio, etc) and decides how much gain reduction should be applied
4. Gain is applied to the original signal before it reaches the output

A feedback design uses the same components as a feedforward design, but as you can see below it does the steps in a slightly different order:

![](https://raw.githubusercontent.com/jd-13/dsp-math-for-audio/master/assets/compressors/simpleFeedback.png)

Here the same process has occured as in the feedforward design, but this time we're using the signal after it has already been compressed to drive the level detector.

Most modern compressors are based on a feedforward design as it allows for the implementation of lookahead, and perfect limiting.

Feedback designs were often used in early analogue compressors as they reduce the dynamic range which the level detector and gain computer must be able to operate effectively on, since they only see the input signal after its dynamic range has already been reduced.

In digital compressors this is no longer a concern, so for the remainder of this notebook we will be focusing on feedforward designs.

### Level detection

Since the level detector is the start of the chain, this is where we'll start with our implementaion

If you've read the previous notebooks on envelope followers (part 1 [here](https://colab.research.google.com/github/jd-13/dsp-math-for-audio/blob/master/02_Envelope-Followers.ipynb), and part 2 [here](https://colab.research.google.com/github/jd-13/dsp-math-for-audio/blob/master/03_Envelope-Followers-Part-2.ipynb)) then this section may seem familiar. The process of performing level detection is conceptually very similar to that of an envelope follower - an input signal is provided, and from it another signal describing how it's amplitude changes is produced.

Also like envelope followers they provide attack and release times which can be controlled by the user, which controls how quickly the produced signal rises and falls while tracking the amplitude of the input signal.

There are two common methods of level detection used in compressors: root-mean-squared (RMS) and peak.

* __RMS__: Provides a smoothed average of the signal that more closely represents the perceived loudness, but introduces a latency in order to calculate the average
* __Peak__: Follows the amplitude of the signal without any averaging, this is the most commonly used method in compressors

Now we'll look at two possible implementations, and plot their outputs to compare their performance. Let's first look at a simple implementation of a peak level detector:

In [3]:
def simplePeakLevelDetector(inputSignal, attack, release):
    """
    Example implementation of a basic peak level detection algorithm with
    separate attack and release controls.
    
    Based on equation 12 of the previously linked paper.
    """
    
    outputBuffer = np.zeros(inputSignal.shape)
    
    for index in range(len(inputSignal)):

        outputBuffer[index] = release * outputBuffer[index - 1] \
                              + (1 - attack) * max(inputSignal[index] - outputBuffer[index - 1], 0)
        
    return outputBuffer

The above design is one of the simplest but is still used often. The code is fairly simple, it first creates an empty output buffer of the correct size and runs the following loop for each sample:

1. Multiply the release coefficient by the previously processed sample - a greater release coefficient means this value will be greater and hang around longer, a smaller release coefficient will cause this value to decay faster

2. If the signal is rising, max() will return the difference between the previously processed sample and the current sample, which is then multiplied by the term (1 - attack coefficient) and added to our output, else max() returns 0 and this entire attack term is ignored

This works reasonably well for many applications, but unfortunately it couples the effect of the attack and release coefficents such that they don't operate independently. For example, in this implementation a shorter release effectively also causes the attack to be shorter.

This can be addressed by breaking the attack and release phases into separate branches of an if statement, which brings us to the next implementation:

In [4]:
def decoupledPeakLevelDetector(inputSignal, attack, release):
    """
    Example implementation of a peak level detection algorithm with decoupled
    attack and release constants.
    
    Based on equation 16 of the previously linked paper.
    """

    outputBuffer = np.zeros(inputSignal.shape)
    
    for index in range(len(inputSignal)):
        
        if inputSignal[index] > outputBuffer[index - 1]:
            # Attack phase
            outputBuffer[index] = attack * outputBuffer[index - 1] + (1 - attack) * inputSignal[index]
        else:
            # Release phase
            outputBuffer[index] = release * outputBuffer[index - 1] + (1 - release) * inputSignal[index]
    
    return outputBuffer

As before the first thing is to create our output buffer, but in the for loop we now check if the current input sample is greater than the last processed sample.

If the input is greater, then the signal must be rising and the level detector should treat this as the attack phase. If the input is smaller (or the same), then the signal must be falling (or staying the same) and the level detector should treat this as the release phase.

You can see that the equations in both the attack and release phase are the same, but just using their respective attack and release coefficients.

For the attack phase, we do the following two things:
1. Multiply the attack coefficient by the previously processed sample - a greater attack coefficient means this value will be greater and therefore the previously processed samples have more influence on the processed output for this sample and future ones
2. Multiply (1 - attack coefficient) by the current input sample and add it to the output - here a greater attack coefficient means this value will be smaller and the current input sample has less influence on the current output sample

The release phase works in exactly the same way, just using the release coefficient.

The two steps described above mean that at high attack/release coefficients the level detector will lag behind the input because previous samples have more influence on the output than current ones, and at smaller attack/release coefficients the level detector will track the input more closely as the current input sample becomes the dominating influence on the output.

One last thing to note is that both of these implementations expect the signal to have been rectified before being passed to them. This means that the signal will contain only positive values, and we need to do this because a level detector only cares about the absolute magnitude of the signal, not whether it is positive or negative.

Now that we've seen both implementations, let's see how they compare when processing a simple signal:

In [5]:
@interact(attack=(0.1, 1, 0.05), release=(0.5, 1, 0.05))
def plotLevelDetector(attack, release):
    """
    Feeds a sine wave impluse to the level detectors and plots the outputs.
    """
    
    # Generate the input signal
    signalBuffer = sineWaveDecay(3000)

    # Plot the input signal
    plt.figure(figsize=(15, 4))
    plt.plot(SAMPLE_RANGE, signalBuffer, label="Input")
    
    # Rectify the signal using abs()
    absBuffer = abs(signalBuffer)
    
    # Plot the simple level detector
    simpleLevelBuffer = simplePeakLevelDetector(absBuffer, attack, release)
    plt.plot(SAMPLE_RANGE, simpleLevelBuffer, label=f"Simple Peak Detector")

    # Plot the decoupled level detector
    decoupledLevelBuffer = decoupledPeakLevelDetector(absBuffer, attack, release)
    plt.plot(SAMPLE_RANGE, decoupledLevelBuffer, label=f"Decoupled Peak Detector")

    # Configure the plot
    plt.ylim(bottom=-Y_LIMIT, top=Y_LIMIT)
    plt.xlabel("Samples")
    plt.ylabel("Displacement")
    plt.legend(loc='upper left')

    plt.show()

interactive(children=(FloatSlider(value=0.55, description='attack', max=1.0, min=0.1, step=0.05), FloatSlider(value=0.75, description='release', max=1.0, min=0.5, step=0.05), Output()), _dom_classes=('widget-interact',))

The most noticable difference is in how the simple detector produces a much smaller output than the decoupled detector except for at the fastest attack and slowest release, whereas the decoupled detector tracks the input much more closely.

Also notice how when changing the release, the attack of the simple detector changes but the attack of the decoupled detector does not. This demonstrates that the decoupled detector does indeed decouple the effects of its attack and release coeffcients.

### Gain computer

The gain computer is the component which takes the output signal from the level detector, and uses the threshold, ratio, and knee parameters to decide how much gain (reduction) to apply to the original input signal.

The **knee** parameter softens the transition when the signal starts to approach the threshold and compression is applied. This might be a little hard to imagine but we'll see an example shortly.

Now that we know what the gain computer is supposed to do, we can look at an example implementation and see how it works:

In [6]:
def gainComputer(inputBuffer, thresholddB, ratio, kneeWidth):
    """
    Simple gain computer with adjustable knee width based on equations 4 and 20 of the linked paper.
    """

    outputBuffer = np.zeros(inputBuffer.shape)

    for index in range(len(inputBuffer)):
        if inputBuffer[index] - thresholddB < -kneeWidth / 2:
            # Level is sufficiently far below the threshold to be unaffected by the knee
            # No compression applied, so output equals input
            outputBuffer[index] = inputBuffer[index]
            
        elif abs(inputBuffer[index] - thresholddB) <= kneeWidth / 2:
            # Level is close enough to the threshold to be affected by the knee
            # Some compression is applied but at a lower ratio
            outputBuffer[index] = inputBuffer[index] + (1 / ratio - 1) \
                                  * pow(inputBuffer[index] - thresholddB + kneeWidth / 2, 2) / (2 * kneeWidth)

        else:
            # Level is sufficiently far above the threshold to be unaffected by the knee
            # Compress the signal above the threshold by the ratio
            outputBuffer[index] = thresholddB + (inputBuffer[index] - thresholddB) / ratio

    return outputBuffer

As in the level detector we first create an output buffer, then we go through our input buffer, sample by sample. This input buffer contains the output from our level detector, after it has been converted to decibels. When we convert the values from the level detector from it's linear scale between 0 and 1 to the logarithmic decibel scale, 0 becomes -infinity dB, and 1 becomes 0dB.

Since negative infinity isn't very useful to actually do maths with, we set a lower bound at -100dB. We also use the threshold value in decibels, since this will be subtracted from the input so needs to be in the same units.

After the output of the gain computer we'll later add another stage to convert from decibels back to a linear value between 0 and 1, which we can multiply by our input signal to apply the compression.

All the important processing here happens inside the for loop, which you can see contains an if-else with three branches:

#### Level is below threshold

This is pretty simple, no compression is applied, so output equals input.

Since the compressor has a knee we need to make sure that the level is sufficiently low that it doesn't enter this transitional region, so we first get the distance between the threshold and the level, and compare this to half of the knee width.

We only use half of the knee width since the knee width extends either side of the threshold, so when comparing to a distance from the threshold we only need to use half of it.

#### Level is near threshold

When the level gets near to the threshold we need to start applying *some* compression but not the full amount. At this point the level could be above or below the threshold, but as long as it is in the transitional region the same math can be applied either way. This is why we use abs() when checking the distance from the threshold, and again why we divide the knee width by 2.

The maths in this branch might look a little more complex, but it will make sense after some thought. Let's start by considering what will happen when the input level is only *just* high enough to be affected by the knee, so sits at the lower boundary of the knee. In this case the following is true:

$ input level = threshold - \frac{knee width}{2} $

If we substitute that into the equation used by this branch of the if statement, we see that it cancels out with other terms and leaves only the input level term:

$ output = input level + (\frac{1}{ratio} - 1)(input level - threshold + \frac{knee width}{2})^2\frac{1}{2 * knee width} $

$ output = input level + (\frac{1}{ratio} - 1)((threshold - \frac{knee width}{2}) - threshold + \frac{knee width}{2})^2\frac{1}{2 * knee width} $

$ output = input level + (\frac{1}{ratio} - 1)(0)^2\frac{1}{2 * knee width} $

$ output = input level $

This makes sense, because at the lower boundary of the knee we would expect it to meet smoothly with the equation for the region below the threshold and knee, which is simply output equals input.

Now for the other end of this equation, when the input level is not *quite* high enough to escape the knee, so sits at the upper boundary of the knee. In this case the following is true:

$ input level = threshold + \frac{knee width}{2} $

Now let's substitute this into the equation for this branch as we did before:

$ output = input level + (\frac{1}{ratio} - 1)(input level - threshold + \frac{knee width}{2})^2\frac{1}{2 * knee width} $

$ output = input level + (\frac{1}{ratio} - 1)((threshold + \frac{knee width}{2}) - threshold + \frac{knee width}{2})^2\frac{1}{2 * knee width} $

$ output = input level + (\frac{1}{ratio} - 1)knee width^2\frac{1}{2 * knee width} $

$ output = input level + (\frac{1}{ratio} - 1)\frac{knee width}{2} $

Now at this point it's not exactly clear how to simplify this such that it matches the equation for the branch above the knee, so we'll substitute the same value of input level into that equation for the upper branch and attempt to meet in the middle:

$ output = threshold + \frac{input level - threshold}{ratio} $

$ output = input level - \frac{knee width}{2} + \frac{(threshold + \frac{knee width}{2}) - threshold}{ratio} $

$ output = input level - \frac{knee width}{2} + \frac{\frac{knee width}{2}}{ratio} $

$ output = input level + (\frac{1}{ratio} - 1)\frac{knee width}{2} $

As we can see, the equation for the middle branch does simplify to match the equation for the upper branch when at an input level that sits a their boundary.

So now we know that this middle branch in the code meets both the lower and the upper branches of the code at exactly the same values at its boundaries, and in between them the use of the $ x^2 $ term (`pow()` in the code) provides a curve which transitions between the two.

#### Level is above threshold

If the level isn't way below the threshold, and isn't in the transition covered by the knee, then it must be far enough above the threshold to have the full compression ratio applied.

The maths here is relatively simple, we know the level is above the threshold so we get that much for free, then we add to that everything above the threshold after it has been divided by the ratio.

#### Plotting the gain computer

Now that we know how the gain computer works, lets plot it's output to get a better intuition for how it works in practice.

In [7]:
@interact(thresholddB=(-20, 0, 2), ratio=(1, 8, 1), kneeWidth=(1, 10, 1))
def plotGainComputer(thresholddB, ratio, kneeWidth):
    """
    Feeds a range of values from -100dB to 0dB to the gain computer and plots the output.
    """
    
    # Generate the input values
    inputValues = np.linspace(0, 1, 50)
    
    # Plot the input values
    plt.figure(figsize=(6, 4))
    plt.plot(inputValues, inputValues, label="Input")
    
    # Process the input
    inputValuesdB = convertTodB(inputValues)
    gainValuesdB = gainComputer(inputValuesdB, thresholddB, ratio, kneeWidth)
    gainValues = convertToLinear(gainValuesdB)

    # Plot the gain values
    plt.plot(inputValues, gainValues, label="Gain")

    # Configure the plot
    plt.xlim(left=0, right=1)
    plt.ylim(bottom=0, top=1)
    plt.xlabel("Input level")
    plt.ylabel("Output level")
    plt.legend(loc='upper left')

    plt.show()

interactive(children=(IntSlider(value=-10, description='thresholddB', max=0, min=-20, step=2), IntSlider(value=4, description='ratio', max=8, min=1), IntSlider(value=5, description='kneeWidth', max=10, min=1), Output()), _dom_classes=('widget-interact',))

As expected we can see that for input levels far below the threshold the output matches the input exactly, at input levels far above the threshold the full compression ratio is applied, and the knee provides a smooth transition between the two.

### Putting it together

We've now seen all the major components of a compressor individually, so it's time to look at how they are connected and work together. Below we can see a simple implemenation which runs through the feedforward architecture we described before:

In [8]:
def simpleCompressor(inputSignal, attack, release, thresholddB, ratio, kneeWidth):
    # Rectify the signal using abs()
    absBuffer = abs(inputSignal)
    
    # Level detector
    detectorOutput = decoupledPeakLevelDetector(absBuffer, attack, release)
    
    # Convert to dB
    dbOutput = convertTodB(detectorOutput)
    
    # Gain computer
    gainComputerOutput = gainComputer(dbOutput, thresholddB, ratio, kneeWidth)
    
    # Subtract gain computer input from its output
    differenceOutput = gainComputerOutput - dbOutput
    
    # Convert back to linear
    linearOutput = convertToLinear(differenceOutput)
    
    # Apply the gain
    outputBuffer = inputSignal * linearOutput
    
    return linearOutput, outputBuffer

Most of this should make sense from the comments labelling each step, but one noteworthy step is that which subtracts the input to the gain computer from its output.

This is an important step which, after converting back to a linear value, actually provides us with a number smaller than or equal to one by which we can multiply the original input signal. If it's not clear why this works, remember that the gain computer outputs a value which will be smaller than or equal to the output from the level detector.

It doesn't actually matter what this value is in absolute terms, but rather how much smaller it is than the original input to the gain computer. Also note that when the input and output of the gain computer is the same, this subtraction will cancel out and give us a value of 0dB, which when converted to linear is 1 - this should make sense because if the input and output is the same then there has been no gain reduction.

There is however a small problem with this design - when in the release phase, rather than falling smoothly back to a lower level, the release falls rather sharply and then stops when it reaches the level of the input signal.

This can be resolved by rearranging some of the components, such that the level detector is after both the conversion to decibels, and also the gain computer:

In [9]:
def logDomainCompressor(inputSignal, attack, release, thresholddB, ratio, kneeWidth):
    # Rectify the signal using abs()
    absBuffer = abs(inputSignal)
    
    # Convert to dB
    dbOutput = convertTodB(absBuffer)
    
    # Gain computer
    gainComputerOutput = gainComputer(dbOutput, thresholddB, ratio, kneeWidth)
    
    # Subtract the gain computer output
    differenceOutput = dbOutput - gainComputerOutput
    
    # Level detector
    detectorOutput = decoupledPeakLevelDetector(differenceOutput, attack, release)
    
    # Convert back to linear
    linearOutput = convertToLinear(-detectorOutput)
    
    # Apply the gain
    outputBuffer = inputSignal * linearOutput
    
    return linearOutput, outputBuffer

This is our final compressor design for this notebook. To summarise all the pieces we've put together: this design provides decoupled attack and release parameters, a knee parameter, and smooth transitions from attack to release.

Now let's compare it to the previous design by plotting their outputs after processing a simple test signal:

In [10]:
@interact(attack=(0.1, 1, 0.05), release=(0.5, 1, 0.05), thresholddB=(-60, 0, 5), ratio=(1, 8, 1), kneeWidth=(0, 10, 1), showSimpleGain=True, showSimpleOutput=True, showLogGain=True, showLogOutput=True)
def plotCompressor(attack, release, thresholddB, ratio, kneeWidth, showSimpleGain, showSimpleOutput, showLogGain, showLogOutput):
    """
    Feeds a sine wave impluse to both compressor designs and plots the outputs.
    """
    
    # Generate the input signal
    signalBuffer = sineWaveDecay(3000)

    # Plot the input signal
    plt.figure(figsize=(15, 4))
    plt.plot(SAMPLE_RANGE, signalBuffer, label="Input")
    
    # Plot the simple compressor
    simpleCompGain, simpleCompOutput = simpleCompressor(signalBuffer, attack, release, thresholddB, ratio, kneeWidth)
    
    if showSimpleGain:
        plt.plot(SAMPLE_RANGE, simpleCompGain, label=f"Simple Compressor Gain")
    
    if showSimpleOutput:
        plt.plot(SAMPLE_RANGE, simpleCompOutput, label=f"Simple Compressor Output")
    
    # Plot the log domain compressor 
    logCompGain, logCompOutput = logDomainCompressor(signalBuffer, attack, release, thresholddB, ratio, kneeWidth)
    
    if showLogGain:
        plt.plot(SAMPLE_RANGE, logCompGain, label=f"Log Compressor Gain")
    
    if showLogOutput:
        plt.plot(SAMPLE_RANGE, logCompOutput, label=f"Log Compressor Output")

    # Configure the plot
    plt.xlabel("Samples")
    plt.ylabel("Displacement")
    plt.legend(loc='upper left')

    plt.show()
    

interactive(children=(FloatSlider(value=0.55, description='attack', max=1.0, min=0.1, step=0.05), FloatSlider(value=0.75, description='release', max=1.0, min=0.5, step=0.05), IntSlider(value=-30, description='thresholddB', max=0, min=-60, step=5), IntSlider(value=4, description='ratio', max=8, min=1), IntSlider(value=5, description='kneeWidth', max=10), Checkbox(value=True, description='showSimpleGain'), Checkbox(value=True, description='showSimpleOutput'), Checkbox(value=True, description='showLogGain'), Checkbox(value=True, description='showLogOutput'), Output()), _dom_classes=('widget-interact',))

As you can see, the gain reduction applied by the log domain compressor starts its attack slightly ahead of the simple compressor, and its release is initially a little faster but has a much smoother tail. This gives it a more natural sound with less distortion of the input signal.

Now that we've reached the end of the notebook I really recommend adjusting some of the parameters and watching how the shape of the gain reduction curves change, this should provide a better intuition of how these two algorithms are performing.

You may also wish to make some of your own adjustments to the code and run the cells again to see how the behaviour changes, or add a few extra plot() or print() statements to learn more about particular parts of the code.

## Conclusion

In this notebook we have seen how compressors are used and how they can be implemented using a few mathematical concepts that I hope now make sense. The reasoning behind a few of the common design desisions and their impacts on the performance of a compressor should also now be a little clearer.

The log domain compressor implementation demonstrated here is not necessarily the best design, as there are plenty of other approaches which have their own performance characteristics that may suit your use case better. It does however provide a reasonable starting point for a first implementation, and may do just fine depending on the goals of your project.

If you are looking for further ideas to study in this domain I would recommend looking into a few of the other compressor based devices that were mentioned earlier in this notebook, and also having a look through some of the further reading resources listed below. There is also a whole world of analogue modelling which is often applied to digital compressors which maybe of interest.

## Further Reading

Below are several resources that I found useful while researching this notebook. If you wish to read more about this topic, these may be good places to start.

### Texts

[Dynamic Range Compression](https://en.wikipedia.org/wiki/Dynamic_range_compression)  
[Teletronix LA2A - History and How it Works](https://recording.org/threads/teletronix-la2a-history-and-how-it-works.58172)  
[Digital Dynamic Range Compressor Design—A Tutorial and Analysis](https://www.researchgate.net/publication/277772168_Digital_Dynamic_Range_Compressor_Design-A_Tutorial_and_Analysis)  

### Videos

[How To Use A Compressor Pedal](https://www.youtube.com/watch?v=j4NrWQljyso)  